In [213]:
import sys

sys.path.append("../")

import pandas as pd
import numpy as np
import datetime
import os
from pprint import pprint
import matplotlib.pyplot as plt
import time
import vectorbtpro as vbt
from time import time
import helpers as pth
import platform
from dotenv import load_dotenv
import scipy.stats as stats
import time

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 20)

plt.style.use("classic")
# plt.rcParams["axes.grid"] = True
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.formatter.useoffset"] = False
plt.rcParams["axes.formatter.limits"] = [-1000000000, 1000000000]


if platform.system().lower() == "windows":
    base_data_path = "H:\\phitech-data\\01_raw"
else:
    from core_chains.simple.llm import make_Q_chain

    base_data_path = "../../phitech-data/01_raw"
    load_dotenv("../../sandatasci-core/credentials")
    Q = make_Q_chain("gpt-4o-instance1", __vsc_ipynb_file__)

In [214]:
%%html
<style>
.dataframe {
    font-size: 9pt; /* Adjust font size as needed */
}
</style>

In [215]:
I = pth.load_instruments(base_path=base_data_path)
I.keys()

dict_keys(['6B', 'MES'])

In [216]:
pd.Series([.5 for _ in range(128)], name='something').vbt.returns.total()

np.float64(3.46482384157094e+22)

In [217]:
from vectorbtpro.base.accessors import BaseAccessor

series = pd.Series(np.random.normal(0, 1, size=10))
series.vbt.make_symmetric()

,0,1,2,3,4,5,6,7,8,9
0,1.325819,-1.257599,-0.974638,0.240731,-0.899906,-0.283882,0.371037,-0.583056,0.116771,-1.101155
1,-1.257599,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.974638,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.240731,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.899906,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,-0.283882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.371037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,-0.583056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.116771,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,-1.101155,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [218]:
df = pd.DataFrame({
    'open': [1, 2, 3, 4, 5],
    'high': [2.5, 3.5, 4.5, 5.5, 6.5],
    'low': [0.5, 1.5, 2.5, 3.5, 4.5],
    'close': [2, 3, 4, 5, 6]
}, index=vbt.date_range("2020-01-01", periods=5))
df

,open,high,low,close
2020-01-01,1,2.5,0.5,2
2020-01-02,2,3.5,1.5,3
2020-01-03,3,4.5,2.5,4
2020-01-04,4,5.5,3.5,5
2020-01-05,5,6.5,4.5,6


In [219]:
pf = vbt.Portfolio.from_holding(
    open=df.open,
    high=df.high,
    low=df.low,
    close=df.close
)
pf.value

2020-01-01    100.0
2020-01-02    150.0
2020-01-03    200.0
2020-01-04    250.0
2020-01-05    300.0
Freq: D, dtype: float64

In [260]:
multi_close = pd.DataFrame({
    'ticker1': df.close,
    'ticker2': df.close,
    'ticker3': np.random.normal(df.close.mean(), df.close.std(), df.shape[0])
})
multi_open = pd.DataFrame({
    'ticker1': df.open,
    'ticker2': df.open,
    'ticker3': np.random.normal(df.open.mean(), df.open.std(), df.shape[0])
})
multi_close

,ticker1,ticker2,ticker3
2020-01-01,2,2,4.110148
2020-01-02,3,3,3.758933
2020-01-03,4,4,1.721007
2020-01-04,5,5,4.629392
2020-01-05,6,6,4.687222


In [263]:
vbt.Portfolio.from_holding(close=multi_close).value

,ticker1,ticker2,ticker3
2020-01-01,100.0,100.0,100.000000
2020-01-02,150.0,150.0,91.454935
2020-01-03,200.0,200.0,41.872130
2020-01-04,250.0,250.0,112.633212
2020-01-05,300.0,300.0,114.040219


In [267]:
candle_green = multi_close > multi_open
candle_green.vbt.signals.fshift(1)

,ticker1,ticker2,ticker3
2020-01-01,False,False,False
2020-01-02,True,True,True
2020-01-03,True,True,True
2020-01-04,True,True,False
2020-01-05,True,True,True


In [268]:
candle_red = multi_close < multi_open
candle_red.vbt.signals.fshift(1)

,ticker1,ticker2,ticker3
2020-01-01,False,False,False
2020-01-02,False,False,False
2020-01-03,False,False,False
2020-01-04,False,False,True
2020-01-05,False,False,False


In [380]:
macd = vbt.MACD.run(
    multi_close,
    fast_window=2,
    slow_window=(3, 4, 5, 6),
    signal_window=2,
    macd_wtype="simple",
    signal_wtype="weighted"
)
macd.signal

macd_fast_window          2                                               \
macd_slow_window          3                          4                     
macd_signal_window        2                          2                     
macd_macd_wtype      simple                     simple                     
macd_signal_wtype  weighted                   weighted                     
                    ticker1 ticker2   ticker3  ticker1 ticker2   ticker3   
2020-01-01              NaN     NaN       NaN      NaN     NaN       NaN   
2020-01-02              NaN     NaN       NaN      NaN     NaN       NaN   
2020-01-03              NaN     NaN       NaN      NaN     NaN       NaN   
2020-01-04              0.5     0.5 -0.281961      NaN     NaN       NaN   
2020-01-05              0.5     0.5  0.587874      1.0     1.0  0.512889   

macd_fast_window                                                      
macd_slow_window          5                        6                  
macd_signal_window        2                        2                  
macd_macd_wtype      simple                   simple                  
macd_signal_wtype  weighted                 weighted                  
                    ticker1 ticker2 ticker3  ticker1 ticker2 ticker3  
2020-01-01              NaN     NaN     NaN      NaN     NaN     NaN  
2020-01-02              NaN     NaN     NaN      NaN     NaN     NaN  
2020-01-03              NaN     NaN     NaN      NaN     NaN     NaN  
2020-01-04              NaN     NaN     NaN      NaN     NaN     NaN  
2020-01-05              NaN     NaN     NaN      NaN     NaN     NaN

In [426]:
from pprint import pprint

groups = macd.signal.groupby(['macd_slow_window'], axis=1).groups
for k, v in groups.items():
    print(f'{k} -> {v}')

3 -> MultiIndex([(2, 3, 2, 'simple', 'weighted', 'ticker1'),
            (2, 3, 2, 'simple', 'weighted', 'ticker2'),
            (2, 3, 2, 'simple', 'weighted', 'ticker3')],
           names=['macd_fast_window', 'macd_slow_window', 'macd_signal_window', 'macd_macd_wtype', 'macd_signal_wtype', None])
4 -> MultiIndex([(2, 4, 2, 'simple', 'weighted', 'ticker1'),
            (2, 4, 2, 'simple', 'weighted', 'ticker2'),
            (2, 4, 2, 'simple', 'weighted', 'ticker3')],
           names=['macd_fast_window', 'macd_slow_window', 'macd_signal_window', 'macd_macd_wtype', 'macd_signal_wtype', None])
5 -> MultiIndex([(2, 5, 2, 'simple', 'weighted', 'ticker1'),
            (2, 5, 2, 'simple', 'weighted', 'ticker2'),
            (2, 5, 2, 'simple', 'weighted', 'ticker3')],
           names=['macd_fast_window', 'macd_slow_window', 'macd_signal_window', 'macd_macd_wtype', 'macd_signal_wtype', None])
6 -> MultiIndex([(2, 6, 2, 'simple', 'weighted', 'ticker1'),
            (2, 6, 2, 'simple', 'wei

C:\Users\spask\AppData\Local\Temp\ipykernel_28340\1088083385.py:3: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  groups = macd.signal.groupby(['macd_slow_window'], axis=1).groups


In [439]:
series = dict(
    close=pd.DataFrame({  
        'a': [1, 2, 3, 4], 
        'b': [4, 3, 2, 1]
    }),
    size=pd.Series([1, -1, 1, -1]),  
    direction=[['longonly', 'shortonly']],  
    fees=0.01  
)
bcast = vbt.broadcast(series)

In [441]:
bcast['close']

,a,b
0,1,4
1,2,3
2,3,2
3,4,1


In [443]:
bcast['fees']

,a,b
0,0.01,0.01
1,0.01,0.01
2,0.01,0.01
3,0.01,0.01


In [445]:
fast_ma = vbt.MA.run(multi_close, window=[2,3], short_name='fast')
slow_ma = vbt.MA.run(multi_close, window=[3,4], short_name="slow")

In [447]:
fast_ma.ma

fast_window       2                         3                  
            ticker1 ticker2   ticker3 ticker1 ticker2   ticker3
2020-01-01      NaN     NaN       NaN     NaN     NaN       NaN
2020-01-02      2.5     2.5  3.934541     NaN     NaN       NaN
2020-01-03      3.5     3.5  2.739970     3.0     3.0  3.196696
2020-01-04      4.5     4.5  3.175199     4.0     4.0  3.369777
2020-01-05      5.5     5.5  4.658307     5.0     5.0  3.679207

In [449]:
slow_ma.ma

slow_window       3                         4                  
            ticker1 ticker2   ticker3 ticker1 ticker2   ticker3
2020-01-01      NaN     NaN       NaN     NaN     NaN       NaN
2020-01-02      NaN     NaN       NaN     NaN     NaN       NaN
2020-01-03      3.0     3.0  3.196696     NaN     NaN       NaN
2020-01-04      4.0     4.0  3.369777     3.5     3.5  3.554870
2020-01-05      5.0     5.0  3.679207     4.5     4.5  3.699138

In [458]:
fast_ma.vbt > slow_ma

fast_window       2                       3                
slow_window       3                       4                
            ticker1 ticker2 ticker3 ticker1 ticker2 ticker3
2020-01-01    False   False   False   False   False   False
2020-01-02    False   False   False   False   False   False
2020-01-03     True    True   False   False   False   False
2020-01-04     True    True   False    True    True   False
2020-01-05     True    True    True    True    True   False

In [461]:
multi_close.vbt > slow_ma

slow_window       3                       4                
            ticker1 ticker2 ticker3 ticker1 ticker2 ticker3
2020-01-01    False   False   False   False   False   False
2020-01-02    False   False   False   False   False   False
2020-01-03     True    True   False   False   False   False
2020-01-04     True    True    True    True    True    True
2020-01-05     True    True    True    True    True    True

In [466]:
above_lower = multi_close.vbt > vbt.Param([1, 2], name='lower')
below_upper = multi_close.vbt > vbt.Param([3,4], name='upper')
above_lower

lower            1                       2                
           ticker1 ticker2 ticker3 ticker1 ticker2 ticker3
2020-01-01    True    True    True   False   False    True
2020-01-02    True    True    True    True    True    True
2020-01-03    True    True    True    True    True   False
2020-01-04    True    True    True    True    True    True
2020-01-05    True    True    True    True    True    True

In [475]:
above_lower.vbt & below_upper

lower            1                       2                
upper            3                       4                
           ticker1 ticker2 ticker3 ticker1 ticker2 ticker3
2020-01-01   False   False    True   False   False    True
2020-01-02   False   False    True   False   False   False
2020-01-03    True    True   False   False   False   False
2020-01-04    True    True    True    True    True    True
2020-01-05    True    True    True    True    True    True